# MTOW

This section describes the code used for computing the maximum takeoff weight for a given airplane parameters. The MTOW is computed using an iterative method similar to [initial weight estimation](../initial_weight_estimation.ipynb#mtow-estimation) but with refined methods. At start, MTOW is set to a random guess value. For this MTOW, the fuel and empty weight is computed using ``compute_fuel_weight`` and ``compute_empty_weight`` function defined earlier. Then, the total weight is estimated and compared against the guess value. This iterative process continues until the difference between computed and guessed MTOW is below a specified tolerance. Below code block defines a function for performing this iterative process:

In [1]:
import nbimporter
import numpy as np
from parameters import AircraftParameters
from fuel import compute_fuel_weight
from empty_weight import compute_empty_weight

def compute_takeoff_weight(wing_loading, power_loading, A):
    """
        Function to compute takeoff weight for a given wing and power loading

        Parameters
        ----------
        wing_loading: float
            wing loading of the airplane (lbs/sq ft)

        power_loading: float
            power loading of the airplane (lbs/hp)

        aspect ratio: float
            aspect ratio of the airplane

        Returns
        -------
        MTOW: float
            converged maximum takeoff weight

        aircraft: AircraftParameters
            parameter object for the converged airplane
    """

    # Variables
    error = np.inf
    tolerance = 0.01
    wc = 200 # lbs, crew
    wp = 1000 # lbs, passengers
    itr = 0
    max_itr = 100
    wto_guess = 5000

    while error > tolerance and itr < max_itr:

        S = wto_guess / wing_loading # sq ft
        takeoff_power = wto_guess / power_loading # hp
        
        # initialize aircraft with updated S and P
        aircraft = AircraftParameters(A, S, takeoff_power) 

        # Fuel weight
        wfuel = compute_fuel_weight(aircraft, wto_guess)

        # Empty weight
        wempty = compute_empty_weight(aircraft, wto_guess)

        # MTOW
        wto = wc + wp + wempty + wfuel

        # Error
        error = np.abs(wto - wto_guess)

        # Guess for next iteration
        wto_guess = wto

        # Increment the counter
        itr += 1

    return wto, aircraft

Below code block demonstrates how the takeoff weight function can be used:

In [2]:
A = 8
wing_loading = 40 # lbs/sq ft
power_loading = 9.25 # lbs/hp

MTOW, final_parameters = compute_takeoff_weight(wing_loading, power_loading, A)

print(f"Maximum takeoff weight for given A, W/S, and W/P: {MTOW:.0f} lbs")
print(f"Empty weight: {final_parameters.empty_weight:.0f} lbs")
print(f"Fuel weight: {final_parameters.fuel_weight:.0f} lbs")
print(f"Wing reference area: {final_parameters.S:.0f} sq ft")
print(f"Takeoff power: {final_parameters.P_takeoff:.0f} hp")

Maximum takeoff weight for given A, W/S, and W/P: 6497 lbs
Empty weight: 3518 lbs
Fuel weight: 1779 lbs
Wing reference area: 162 sq ft
Takeoff power: 702 hp


This concludes the takeoff weight estimation section. The next section consists of functions for computing performance constraints.